# Pre-procesamiento de datos - RD
Crearemos una tabla con los datos necesarios para utilizar un algoritmo de clasificación simple

In [2]:
# Importamos las librerías a utilizar
import h5py                             # Para manejar los archivos .h5
import pyjet as fj                      # Clustering de los jets
import numpy as np
import pandas as pd                     # Manejo de tablas
import os.path                          # Manejo de directorios
from os import path
from tqdm import tqdm                   # Barra de progreso

In [ ]:
# Cargamos el archivo con los datos como un dataframe
# El archivo se puede descargar en https://zenodo.org/record/4536624
df = pd.read_hdf("../../events_anomalydetection.h5")

In [3]:
# Escogemos el número de eventos a analizar
n_eventos = 100000

In [ ]:
# Hacemos un sample random de estos eventos
dfsample = df.sample(n=n_eventos)

# Los guardamos en un archivo .h5 para tener la muestra en la que se haga el análisis
if path.exists("../data/events_anomalydetection_tiny_{}.h5".format(n_eventos))!= True: 
    dfsample.to_hdf("../data/events_anomalydetection_tiny_{}.h5".format(n_eventos), key='df', mode='w',complevel=5,complib='blosc')

In [4]:
# Cargamos la muestra en un dataframe (en caso de tener el archivo no hace falta correr la celda anterior)
eventos_tiny = pd.read_hdf("../data/events_anomalydetection_tiny_{}.h5".format(n_eventos))

In [5]:
# Verificamos el tamaño (n_eventos x nro. hadrones*3(caracteristicas)+1(señal))
eventos_tiny.shape

(100000, 2101)

### Funciones para calcular variables

Calcularemos las siguientes varibles:
#### La distancia angular entre dos jets

$$
\Delta R = \sqrt{(\phi_1-\phi_2)^2+(\eta_1-\eta_2)^2}
$$
  
#### N-Subjettiness

El 0-, 1- y 2-subjettiness están definidos como:

$$
\begin{align}
\tau_0(\beta) &= \sum_{i\in J} p_{T_i}\Delta R^\beta \\
\tau_1(\beta) &= \frac{1}{\tau_0(\beta)} \sum_{i\in J} p_{T_i}\Delta R_{a_1,i}^\beta \\
\tau_2(\beta) &= \frac{1}{\tau_0(\beta)}\sum_{i\in J} p_{T_i} \text{min}(\Delta R_{a1,i}^\beta,\Delta R_{a_2,i})
\end{align}
$$
    Para generar una variable adimensional:
$$
\tau_{21}=\frac{\tau_2}{\tau_1}
$$

In [6]:
def deltaR(x, y):
    return ((x.phi-y.phi)**2 + (x.eta-y.eta)**2)**0.5

def subjettiness(cndts, cnsts):
    d0 = sum(c.pt for c in cnsts)
    ls = []
    for c in cnsts:
        dRs = [deltaR(c,cd) for cd in cndts]
        ls += [c.pt * min(dRs)]
    return sum(ls)/d0

def tau21(jet,subR=0.2):
    '''Input: jet from the jet clustering result '''
    jet_substruct_features = {}        
    seq = fj.cluster(jet, R=subR, algo='kt')
    cnsts = jet.constituents()
    cndts1 = seq.exclusive_jets(1)
    tau1 = subjettiness(cndts1, cnsts)
    if (len(cnsts)>1):
        cndts2 = seq.exclusive_jets(2)
        tau2 = subjettiness(cndts2, cnsts)
    else: 
        tau2 = 0
    
    try:
        return tau2/tau1
    
    except ZeroDivisionError:
        return 0

### Clustering y tabla
Haremos el clustering de los datos de manera separada para señal y para fondo, por lo que crearemos una función que haga el agrupamiento y genere una tabla. La tabla contendrá $p_T$, $m$, $\eta$, $\phi$, $E$, $\tau_{21}$, $m_{jj}$ y $\Delta R_{12}$ de los dos jets principales, así como el *nro. de hadrones* del evento.

In [7]:
def tabla(datos):
    n_eventos = datos.shape[0]                  # número de eventos (1000)
    n_hadrones_gen = int((datos.shape[1]-1)/3)  # número de hadrones (700) 
                                                # [-1 para eliminar la columna señal, /3 por las 3 caracteristicas de cada hadron]
    datos_ss = datos.iloc[:,:-1]                # El dataframe pero sin la señal
    
    df = pd.DataFrame(columns=['pT_j1', 'm_j1', 'eta_j1', 'phi_j1', 'E_j1', 'tau_21_j1',  
                                'pT_j2', 'm_j2', 'eta_j2', 'phi_j2', 'E_j2', 'tau_21_j2',
                                'm_jj', 'deltaR_j12', 'label'])

    for evento in tqdm(range(n_eventos)):

        pseudojets_input = np.zeros(len([data for data in datos_ss.iloc[evento,::3] if data > 0]), dtype= fj.DTYPE_PTEPM) 

        for hadron in range(n_hadrones_gen):
            if (datos_ss.iloc[evento,hadron*3] > 0): ## si pT > 0 

                ## Llenamos el arreglo con pT, eta y phi de cada "partícula"
                pseudojets_input[hadron]['pT'] = datos_ss.iloc[evento,hadron*3] 
                pseudojets_input[hadron]['eta'] = datos_ss.iloc[evento,hadron*3+1]
                pseudojets_input[hadron]['phi'] = datos_ss.iloc[evento,hadron*3+2]

                pass
            pass

        ## Devuelve una "ClusterSequence" (un tipo de lista de pyjet)
        secuencia = fj.cluster(pseudojets_input, R=1.0, p=-1) 

        ## Con inclusive_jets accedemos a todos los jets que fueron clusterizados
        ## y filtramos los que tienen pT mayor que 20.
        ## Hacemos una lista con objetos PseudoJet
        jets = secuencia.inclusive_jets(ptmin=20) 

        # Extraemos las variables de interes del jet principal
        pT_j1 = jets[0].pt
        m_j1 = np.abs(jets[0].mass)
        eta_j1 = jets[0].eta
        phi_j1 = jets[0].phi
        E_j1 = jets[0].e
        tau_21_j1= tau21(jets[0])

        # Intentamos extraer las variables del jet secundario
        try:
            pT_j2 = jets[1].pt
            m_j2 = np.abs(jets[1].mass)
            eta_j2 = jets[1].eta
            phi_j2 = jets[1].phi
            E_j2 = jets[1].e
            tau_21_j2= tau21(jets[1])
    
        # Si no hay jet secundario colocamos ceros
        except IndexError:
            pT_j2 = 0
            m_j2 = 0
            eta_j2 = 0
            phi_j2 = 0
            E_j2 = 0
            tau_21_j2 = 0
        
        # Calculamos las otras variables
        deltaR_j12 = deltaR(jets[0], jets[1])
        mjj = m_j1 + m_j2
        n_hadrones = datos.iloc[evento,:].astype(bool).sum(axis=0)/3
        label = datos.iloc[evento,-1]

        # Agregamos todo al dataframe
        entry = pd.DataFrame([[pT_j1, m_j1, eta_j1, phi_j1, E_j1, tau_21_j1,  
                                pT_j2, m_j2, eta_j2, phi_j2, E_j2, tau_21_j2, 
                                mjj,deltaR_j12, n_hadrones, label]],
                            columns=['pT_j1', 'm_j1', 'eta_j1', 'phi_j1', 'E_j1', 'tau_21_j1',  
                                    'pT_j2', 'm_j2', 'eta_j2', 'phi_j2', 'E_j2', 'tau_21_j2',
                                    'm_jj', 'deltaR_j12', 'n_hadrones', 'label'])
        df = df.append(entry, sort=True)
    return df

Generamos la tabla:

In [8]:
df = tabla(eventos_tiny)
df.head()

100%|██████████| 100000/100000 [55:44<00:00, 29.90it/s] 


,E_j1,E_j2,deltaR_j12,eta_j1,eta_j2,label,m_j1,m_j2,m_jj,n_hadrones,pT_j1,pT_j2,phi_j1,phi_j2,tau_21_j1,tau_21_j2
0,1541.103551,1314.832020,3.046675,-0.676879,-0.479236,0.0,140.326113,53.455092,193.781205,212.0,1239.698604,1176.086061,-0.949919,2.090338,0.672299,0.556964
0,1735.554053,1456.732975,3.175913,0.487036,0.215936,0.0,140.960881,91.964940,232.925821,160.0,1543.152632,1420.578669,0.580209,-2.584112,0.429885,0.709729
0,1747.277774,2131.876916,3.196916,-0.561281,-0.915332,0.0,130.198271,595.900967,726.099237,245.0,1499.892043,1412.636020,-2.168662,1.008589,0.682710,0.489799
0,1955.619525,2206.429386,3.483775,0.608165,-0.867853,0.0,388.869494,37.471626,426.341119,116.0,1609.609763,1574.869196,-0.565932,2.589707,0.740089,0.559426
0,1810.986292,2424.718009,3.642805,-0.664116,1.154562,0.0,242.616470,201.110818,443.727289,146.0,1460.557601,1385.597608,-1.685001,1.471332,0.437715,0.566019


Guardamos el dataframe como un csv:

In [9]:
outname = 'eventosRD_{}.csv'.format(n_eventos)
outdir = '../data'
# Si no existe la carpeta, la crea
if not os.path.exists(outdir):
    os.makedirs(outdir)

path = os.path.join(outdir, outname)    
df.to_csv(path, sep=',', index=False)